In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import geopandas as gpd
from matplotlib.colors import TwoSlopeNorm  # Use CenteredNorm for newer matplotlib versions
from matplotlib.cm import ScalarMappable
import matplotlib.colors

In [31]:
matplotlib.rc('font', family='Arial') 

In [41]:
# Create a new figure with custom subplot layout
start_year, middle_year1,middle_year2, end_year=2003, 2011, 2011, 2019
fig = plt.figure(figsize=(23, 11), dpi=500)

# Create grid specification for subplots
gs = fig.add_gridspec(21, 50)

ax1 = fig.add_subplot(gs[:12, 0:22])
subax1=fig.add_subplot(gs[2:10, 22])
ax1.text(-0.05, 1.05, 'A', transform=ax1.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')

ax2 = fig.add_subplot(gs[:12, 25:47])
subax2 = fig.add_subplot(gs[2:10, 47])
ax2.text(-0.05, 1.05, 'B', transform=ax2.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')
plotAB(ax1, ax2, subax1, subax2 , start_year, middle_year1,middle_year2, end_year)
ax3 = fig.add_subplot(gs[12:, 0:10])
ax31 = fig.add_subplot(gs[12:, 10])
ax3.text(-0.05, 1.1, 'C', transform=ax3.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')

ax4 = fig.add_subplot(gs[12:, 13:23])
ax41 = fig.add_subplot(gs[12:, 23])
ax4.text(-0.05, 1.1, 'D', transform=ax4.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')


ax5 = fig.add_subplot(gs[12:, 26:36])
ax51 = fig.add_subplot(gs[12:, 36])

ax5.text(-0.05, 1.1, 'E', transform=ax5.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')



ax6 = fig.add_subplot(gs[12:, 39:49])
ax61 = fig.add_subplot(gs[12:, 49])
ax6.text(-0.05, 1.1, 'F', transform=ax6.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')


Results=CDEF([ax3,ax4,ax5,ax6], start_year, middle_year1,middle_year2, end_year)

plot_subsub([ax31,ax41, ax51, ax61], Results)
# Adjust the layout manually
fig.subplots_adjust(left=0.05, right=0.95, top=1, bottom=0.1, hspace=0.1, wspace=0.3)#plt.tight_layout()
#fig.savefig('Final Figure2-ver3.pdf')

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\4062662442.py:46: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\4062662442.py:97: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\1649085844.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will bec

In [7]:
def plot_subsub(axis, Results):
    for ax, res in zip(axis, Results):
        # Create DataFrame
        df = res.reset_index()
        # Set index to country names for better plotting
        df.set_index('Aff_country', inplace=True)
        norm = matplotlib.colors.LogNorm()  # Normalization for the colormap
        sns.heatmap(df[['sum']], annot=True, fmt=".1f", cmap='viridis', cbar=False, norm=norm, ax=ax)
        ax.set_title('sum')
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_xlabel('')
        ax.set_ylabel('')

In [9]:
def plotAB(ax3, ax4, subax3, subax4, start_year, middle_year1,middle_year2, end_year):  
    df=pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
    df=df.rename(columns={'year':'Year', 'aggregated_value':'count', 'country':'Mention_country', 'affiliation_country':'Aff_country'})
    Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
    abbr=[country.lower() for country in Country_list.values()]
    physical_sciences=[ 'MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
    df=df[~df['subjarea'].isin(physical_sciences)]
    df=df.groupby(['Year', 'Mention_country', 'Aff_country'])['count'].sum().reset_index()
    
    country_codes=pd.read_csv(r"C:\Users\Yasaman\Downloads\iso3.csv")
    country_codes['iso3']=[c.lower() for c in country_codes['iso3']]
    map={country_codes.iloc[c]['iso3']: country_codes.iloc[c]['name'] for c in range(len(country_codes))}
    map['irn']='Iran'
    map['usa']='USA'
    map['gbr']='UK'
    def cdf(df, column):
      data_sorted = df.sort_values(by=column)# Sort the data
      data_sorted['cdf']=np.arange(1, len(data_sorted)+1) / len(data_sorted)
      return data_sorted
    before_df=df[df['Year'].isin(np.arange(start_year, middle_year1, 1))].groupby('Aff_country')['count'].sum().reset_index().rename(columns={'count':'count_before'})
    before_df=before_df.merge(df[(df['Year'].isin(np.arange(start_year, middle_year1, 1)))&(df['Mention_country']!=df['Aff_country'])].groupby(by=[ 'Aff_country'])['count'].sum().reset_index().rename(columns={'count':'external_count_before'}), on='Aff_country', how='inner')
    before_df=before_df.merge(df[(df['Year'].isin(np.arange(start_year, middle_year1, 1)))&(df['Mention_country'].isin(abbr))&(df['Mention_country']!=df['Aff_country'])].groupby(by=[ 'Aff_country'])['count'].sum().reset_index().rename(columns={'count':'target_count_before'}), on='Aff_country', how='inner')

    after_df=df[df['Year'].isin(np.arange(middle_year2, end_year, 1))].groupby('Aff_country')['count'].sum().reset_index().rename(columns={'count':'count_after'})
    after_df=after_df.merge(df[(df['Year'].isin(np.arange(middle_year2, end_year, 1)))&(df['Mention_country'].isin(abbr))&(df['Mention_country']!=df['Aff_country'])].groupby(by=[ 'Aff_country'])['count'].sum().reset_index().rename(columns={'count':'target_count_after'}), on='Aff_country', how='inner')
    after_df=after_df.merge(df[(df['Year'].isin(np.arange(middle_year2, end_year, 1)))&(df['Mention_country']!=df['Aff_country'])].groupby(by=[ 'Aff_country'])['count'].sum().reset_index().rename(columns={'count':'external_count_after'}), on='Aff_country', how='inner')

    ccdf_before=cdf(before_df, 'target_count_before')
    ccdf_after=cdf(after_df, 'target_count_after')
    ccdf_before=ccdf_before.rename(columns={'cdf':'cdf_before'})
    ccdf_after=ccdf_after.rename(columns={'cdf':'cdf_after'})

    new_df=ccdf_before.merge(ccdf_after, on='Aff_country')
    new_df['difference']=(-new_df['cdf_before']+new_df['cdf_after'])

    new_df['relative_difference']=new_df['difference']/(1-new_df['cdf_before'])
    new_df['target_count_before']/=len(np.arange(start_year, middle_year1, 1))
    new_df['target_count_after']/=len(np.arange(middle_year2, end_year, 1))
    new_df['dif_count']=-new_df['target_count_before']+new_df['target_count_after']

    new_df['Aff_country']=new_df['Aff_country'].apply(lambda x: x.upper())
    new_df['external_difference']=new_df['external_count_after']-new_df['external_count_before']
    new_df['difference_count']=new_df['count_after']-new_df['count_before']

    # Load the world map
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Assume new_df is a DataFrame with columns 'Aff_country' and 'ratio'
    # Merge your data with the world map
    world = world.merge(new_df, left_on='iso_a3', right_on='Aff_country', how='left')
    # Plot the map
    world.boundary.plot(ax=ax3, color='k', linewidth=0.02)
    norm = matplotlib.colors.LogNorm()  # Normalization for the colormap
    cmap = plt.cm.viridis  # Colormap
    world.plot(column='dif_count', ax=ax3, cmap=cmap, norm=norm,
               missing_kwds={'color': 'lightgrey'})
    
    sm_ax3 = ScalarMappable(norm=norm, cmap=cmap)
    sm_ax3.set_array([])  # Necessary to 'activate' the ScalarMappable for the colorbar
    cbarax3=plt.colorbar(sm_ax3, cax=subax3, orientation='vertical')
    cbarax3.set_label(r"$\Delta A^*_i$", size=18)  # Adjust fontsize here


    # Find the top 5 countries by ratio
    top_countries = world.dropna(subset=['dif_count']).nlargest(5, 'dif_count')



    # Coordinates in the ocean to place annotations and points (manually specified)
    ocean_points = [
        (-220, -15),  # Example coordinates, you may need to adjust these
        (-220, -30),
        (-220, -45),
        (-220, -60),
        (-220, -75)
    ]
    top_countries.set_index(top_countries.name, inplace=True)
    top_countries.rename(index={'United States of America':'USA', 'United Kingdom':'UK', 'United Arab Emirates':'UAE'}, inplace=True)
    #top_countries=top_countries.reset_index()

    # Plot top countries points with annotations in the ocean
    for idx, (point, row) in enumerate(zip(ocean_points, top_countries.iterrows())):
        _, row = row
        color = cmap(norm(row['dif_count']))  # Get the color from the colormap and normalization
        ax3.scatter(point[0], point[1], color=color, edgecolor='black', s=30)  # Plot point
        ax3.text(point[0]+5, point[1]+2, f"{row['name']}:{row['dif_count']:.1f}",
                 verticalalignment='top', horizontalalignment='left', fontsize=13)

    ax3.set_xticks([])
    ax3.set_yticks([])





    # Load the world map
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    new_df['Aff_country'] = new_df['Aff_country'].apply(lambda x: x.upper())
    # Merge your data with the world map
    world = world.merge(new_df, left_on='iso_a3', right_on='Aff_country', how='left')
    vmin = new_df['relative_difference'].min()
    vmax = new_df['relative_difference'].max()
    vcenter = 0
    norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
    cmap = plt.cm.PRGn   
    # Plot the map
    world.boundary.plot(ax=ax4, color='k', linewidth=0.02)
    world.plot(column='relative_difference', ax=ax4, cmap=cmap, norm=norm,
               missing_kwds={'color': 'lightgrey'})
    sm_ax3 = ScalarMappable(norm=norm, cmap=cmap)
    sm_ax3.set_array([])  # Necessary to 'activate' the ScalarMappable for the colorbar
    cbar_ax3 =plt.colorbar(sm_ax3, cax=subax4, orientation='vertical' )
    cbar_ax3.set_label(r"$\frac{\Delta q(A^*_i)}{1-q(A^*_i)}$", size=18)
 
    # Find the top 5 countries by ratio
    top_countries = world.dropna(subset=['relative_difference']).nlargest(5, 'relative_difference')

    # Coordinates in the ocean to place annotations and points (manually specified)
    ocean_points = [
        (-220, 0),  # Example coordinates, you may need to adjust these
        (-220, -15),
        (-220, -30),
        (-220, -45),
        (-220, -60)
    ]

    # Plot top countries points with annotations in the ocean
    for idx, (point, row) in enumerate(zip(ocean_points, top_countries.iterrows())):
        _, row = row
        color = cmap(norm(row['relative_difference']))  # Get the color from the colormap and normalization
        ax4.scatter(point[0], point[1], color=color, edgecolor='black', s=30)  # Plot point
        ax4.text(point[0] + 5, point[1] + 2, f"{row['name']}:{row['relative_difference']:.2f}",
                 verticalalignment='top', horizontalalignment='left', fontsize=13)

    ax4.set_xticks([])
    ax4.set_yticks([])



In [22]:
def CDEF(axis, start_year, middle_year1,middle_year2, end_year):
    df=pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
    df=df.rename(columns={'year':'Year', 'aggregated_value':'count', 'country':'Mention_country', 'affiliation_country':'Aff_country'})
    Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
    rev_Country_list={Country_list[key]: key for key in Country_list}
    abbr=[country.lower() for country in Country_list.values()]
    country_codes=pd.read_csv(r"C:\Users\Yasaman\Downloads\iso3.csv")
    country_codes['iso3']=[c.lower() for c in country_codes['iso3']]
    map={country_codes.iloc[c]['iso3']: country_codes.iloc[c]['name'] for c in range(len(country_codes))}
    map['irn']='Iran'
    map['usa']='USA'
    map['gbr']='UK'
    Results=[]
    social_sciences=['PSYC', 'SOCI', 'DECI', 'ARTS', 'ECON', 'BUSI']
    health_sciences=['MEDI', 'NURS', 'DENT', 'HEAL', 'VETE']
    life_sciences=['PHAR', 'NEUR', 'AGRI', 'IMMU', 'BIOC']
    physical_sciences=['CHEM', 'EART', 'ENER', 'CENG',  'ENVI']
    categories={'Social sciences':social_sciences,"Health sciences":health_sciences, 'Life sciences':life_sciences ,"Physical sciences":physical_sciences }
    
    for ax, category_dict_key in zip(axis, categories.keys()):
        category=categories[category_dict_key]
        filtered_df=df[df['subjarea'].isin(category)]
        before_df=filtered_df[filtered_df['Year'].isin(np.arange(start_year, middle_year1, 1))].groupby(by=['Mention_country','Aff_country']).sum().reset_index()[['count','Mention_country','Aff_country']].rename(columns={'count':'count_before'})
        after_df=filtered_df[filtered_df['Year'].isin(np.arange(middle_year2, end_year, 1))].groupby(by=['Mention_country','Aff_country']).sum().reset_index()[['count','Mention_country','Aff_country']].rename(columns={'count':'count_after'})

        compare_df=before_df.merge(after_df, how='outer', on=['Mention_country','Aff_country']).fillna(0)
        compare_df['count_after']/=len(np.arange(start_year, middle_year1, 1))
        compare_df['count_before']/=len(np.arange(middle_year2, end_year, 1))
        compare_df['difference']=compare_df['count_after']-compare_df['count_before']
        compare_df=compare_df[(compare_df['Mention_country'].isin(abbr))&(compare_df['Mention_country']!=compare_df['Aff_country'])]
        compare_df['Mention_country']=compare_df['Mention_country'].map(map)
        compare_df['Aff_country']=compare_df['Aff_country'].map(map)
        heatmap_data = compare_df.drop_duplicates().fillna(0).pivot(index='Aff_country', columns='Mention_country', values='difference')
        heatmap_data=heatmap_data.fillna(0)
        for col in heatmap_data.columns:
            if col in heatmap_data.index:
              heatmap_data.loc[col][col]=np.nan
        row_sums = heatmap_data.sum(axis=1)
        
        heatmap_data['sum']=row_sums
        df_sorted = heatmap_data.loc[row_sums.sort_values(ascending=False).index][:10 :]
        Results.append(df_sorted['sum'])
        df_sorted.rename(index={'United States of America':'USA', 'United Kingdom':'UK', 'United Arab Emirates':'UAE'}, columns={'Syrian Arab Republic':'Syria'}, inplace=True)
        custom_order=list(Country_list.keys())
        df_sorted = df_sorted[custom_order]
        sns.heatmap(df_sorted.iloc[:, : -1], ax=ax, annot=True, cbar=False,center=0, cmap='BrBG',fmt='.1f',annot_kws={"fontsize":10})
        ax.set_title(category_dict_key, fontsize=20)
        ax.set_xlabel('Mentioned country', fontsize=14)
        if len(Results)==1:
            ax.set_ylabel('Affiliation country', fontsize=14)
        else:
            ax.set_ylabel('')
            
    return Results




In [11]:
df=pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df.rename(columns={'year':'Year', 'aggregated_value':'count', 'country':'Mention_country', 'affiliation_country':'Aff_country'})
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
country_codes=pd.read_csv(r"C:\Users\Yasaman\Downloads\iso3.csv")
country_codes['iso3']=[c.lower() for c in country_codes['iso3']]
map={country_codes.iloc[c]['iso3']: country_codes.iloc[c]['name'] for c in range(len(country_codes))}
map['irn']='Iran'
map['usa']='USA'
map['gbr']='UK'
Results=[]
social_sciences=['PSYC', 'SOCI', 'DECI', 'ARTS', 'ECON', 'BUSI']
health_sciences=['MEDI', 'NURS', 'DENT', 'HEAL', 'VETE']
life_sciences=['PHAR', 'NEUR', 'AGRI', 'IMMU', 'BIOC']
physical_sciences=['CHEM', 'EART', 'ENER', 'CENG',  'ENVI']
categories={'Social sciences':social_sciences,"Health sciences":health_sciences, 'Life sciences':life_sciences ,"Physical sciences":physical_sciences }

for category_dict_key in categories.keys():
    category=categories[category_dict_key]
    filtered_df=df[df['subjarea'].isin(category)]
    before_df=filtered_df[filtered_df['Year'].isin(np.arange(start_year, middle_year1, 1))].groupby(by=['Mention_country','Aff_country']).sum().reset_index()[['count','Mention_country','Aff_country']].rename(columns={'count':'count_before'})
    after_df=filtered_df[filtered_df['Year'].isin(np.arange(middle_year2, end_year, 1))].groupby(by=['Mention_country','Aff_country']).sum().reset_index()[['count','Mention_country','Aff_country']].rename(columns={'count':'count_after'})

    compare_df=before_df.merge(after_df, how='outer', on=['Mention_country','Aff_country']).fillna(0)
    compare_df['count_after']/=len(np.arange(start_year, middle_year1, 1))
    compare_df['count_before']/=len(np.arange(middle_year2, end_year, 1))
    compare_df['difference']=compare_df['count_after']-compare_df['count_before']
    compare_df=compare_df[(compare_df['Mention_country'].isin(abbr))&(compare_df['Mention_country']!=compare_df['Aff_country'])]
    compare_df['Mention_country']=compare_df['Mention_country'].map(map)
    compare_df['Aff_country']=compare_df['Aff_country'].map(map)
    heatmap_data = compare_df.drop_duplicates().fillna(0).pivot(index='Aff_country', columns='Mention_country', values='difference')
    heatmap_data=heatmap_data.fillna(0)
    for col in heatmap_data.columns:
        if col in heatmap_data.index:
            heatmap_data.loc[col][col]=np.nan
    row_sums = heatmap_data.sum(axis=1)
    
    heatmap_data['sum']=row_sums
    df_sorted = heatmap_data.loc[row_sums.sort_values(ascending=False).index][:10 :]
    Results.append(df_sorted['sum'])
    df_sorted.rename(index={'United States of America':'USA', 'United Kingdom':'UK', 'United Arab Emirates':'UAE'}, columns={'Syrian Arab Republic':'Syria'}, inplace=True)

   

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\1374607786.py:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  heatmap_data.loc[col][col]=np.nan
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\1374607786.py:36: FutureWarning: Chaine

In [18]:
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}


In [21]:
df_sorted

Mention_country,Egypt,Tunisia,Libya,Syria,Yemen,Bahrain,Jordan,Kuwait,Morocco,Oman
Aff_country,,,,,,,,,,
France,0.985567,12.297676,-0.022665,0.874819,0.523501,0.208854,0.468223,-0.075397,5.880372,-1.214110
Saudi Arabia,12.606184,1.319919,0.145866,0.100849,1.250452,0.163541,2.482815,0.221972,0.329032,0.960011
Spain,1.621145,4.441477,0.502152,0.889962,0.292280,0.055870,0.263690,0.100335,10.077937,0.333366
Germany,5.022170,0.428453,-0.413051,1.848833,-0.273342,0.050675,6.109250,-0.128828,3.001285,2.840207
Italy,2.868483,4.386737,1.135130,0.656937,1.209500,0.081143,0.196733,-0.009425,3.025925,-0.000255
UK,1.396952,0.195036,3.763150,-0.020713,0.308978,-0.091793,2.268227,0.958449,1.607176,2.596098
USA,5.355992,0.495190,1.578822,0.503284,0.489893,0.177524,0.843798,0.915554,2.131020,0.404442
Iran,0.192895,0.176472,0.062500,0.965716,0.190476,0.110269,0.245748,0.180730,-0.047585,7.480386
Egypt,NaN,0.600633,1.795332,0.225424,2.111464,-0.133333,0.795953,0.394296,0.174697,1.850759
